In [ ]:
# 1. Instalação das Dependências
# Garantir que todas as bibliotecas necessárias estejam instaladas.
# Adicionamos 'mediapipe' que é essencial para a estimativa de pose.
# O pacote 'tensorflow' por padrão instala a versão otimizada para CPU.

# Removido 'tensorflow-gpu' e 'gdown' já que estamos em CPU e o dataset é baixado com kaggle cli
# A linha abaixo deve ser executada uma vez no terminal ou na célula de notebook para instalar/atualizar
# !pip install tensorflow numpy pandas matplotlib requests scikit-learn imbalanced-learn tqdm kaggle mediapipe opencv-python --upgrade
# !pip install --upgrade h5py Keras

# 2. Configurações Iniciais e Verificação de Hardware
# Importações essenciais para o projeto.
import os
import shutil
import numpy as np
import pandas as pd
import cv2
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, LabelEncoder
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import Input, Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
import tensorflow as tf
from tqdm import tqdm
import zipfile
import json
from pathlib import Path
import subprocess
import time
import requests
import sys # Importado para sys.exit()

# Importar MediaPipe para estimativa de pose
import mediapipe as mp

# Configuração de CPU
# Em um ambiente com apenas CPU, o TensorFlow irá automaticamente utilizar o processador.
print("ℹ️ Nenhuma GPU NVIDIA detectada ou configurada. Todas as operações de Machine Learning utilizarão a CPU.")

# Configurar credenciais do Kaggle
# Isso é crucial para baixar o dataset de exercícios automaticamente.
kaggle_dir = Path.home() / '.kaggle'
kaggle_dir.mkdir(exist_ok=True)

# Você pode obter isso no seu perfil do Kaggle, na seção "API".
kaggle_creds = {
    "username": "leobiondi", # Substitua pelo seu username do Kaggle
    "key": "4bfa4af56e33c61cc4852d2e5f626721" # Substitua pela sua chave API do Kaggle
}

# Salvar credenciais no arquivo kaggle.json
with open(kaggle_dir / 'kaggle.json', 'w') as f:
    json.dump(kaggle_creds, f)

# Adicionar ao PATH para que o comando 'kaggle' possa ser executado
# Isso é importante para que o subprocess.run possa encontrar o comando kaggle.
os.environ['PATH'] += os.pathsep + str(Path.home() / '.local' / 'bin')

print("✅ Ambiente configurado. Credenciais do Kaggle salvas e ambiente preparado para CPU.")


# 3. Definição de Funções Auxiliares para Extração de Features (MediaPipe) e Geração de Erros

# Inicializar o MediaPipe Pose para detecção de landmarks.
# Static_image_mode=False é para processamento de vídeo, True para imagens estáticas.
# Min_detection_confidence e min_tracking_confidence são limiares de confiança.
mp_pose = mp.solutions.pose
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles # Corrigido aqui, estava mp.solutions.drawing_utils novamente

# Função para calcular o ângulo entre três pontos (landmarks)
def calculate_angle(a, b, c):
    """
    Calcula o ângulo em graus entre três pontos (landmarks).
    O ponto 'b' é o vértice do ângulo.
    Args:
        a (list/tuple): Coordenadas (x, y) ou (x, y, z) do primeiro ponto.
        b (list/tuple): Coordenadas (x, y) ou (x, y, z) do ponto central (vértice).
        c (list/tuple): Coordenadas (x, y) ou (x, y, z) do terceiro ponto.
    Returns:
        float: O ângulo em graus.
    """
    a = np.array(a) # Primeiro ponto
    b = np.array(b) # Ponto do vértice
    c = np.array(c) # Terceiro ponto

    # Vetores formados pelos pontos
    ba = a - b
    bc = c - b

    # Produto escalar e normas dos vetores
    cosine_angle = np.dot(ba, bc) / (np.linalg.norm(ba) * np.linalg.norm(bc))
    # Garantir que o valor esteja dentro do domínio de arccos para evitar erros de floating point
    angle = np.arccos(np.clip(cosine_angle, -1.0, 1.0))

    return np.degrees(angle)

# Função para extrair features de pose de uma imagem (ângulos e distâncias)
def extract_pose_features(image_path, pose_detector):
    """
    Extrai um vetor de features de pose (ângulos chave do corpo) de uma imagem.
    Args:
        image_path (str): Caminho para a imagem.
        pose_detector (mp.solutions.pose.Pose): Objeto MediaPipe Pose inicializado.
    Returns:
        numpy.ndarray or None: Um vetor numpy de features (ângulos), ou None se nenhuma pose for detectada.
    """
    try:
        # Lendo a imagem
        image = cv2.imread(image_path)
        if image is None:
            # print(f"⚠️ Não foi possível carregar a imagem: {image_path}") # Comentado para evitar poluir o log
            return None

        # Convertendo a imagem de BGR para RGB (MediaPipe espera RGB)
        image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        # Processando a imagem para estimativa de pose
        results = pose_detector.process(image_rgb)

        # Se houver landmarks de pose detectadas
        if results.pose_landmarks:
            landmarks = results.pose_landmarks.landmark
            
            # Mapeamento das landmarks do MediaPipe para fácil acesso
            # Para o `mp_pose.PoseLandmark` o `.value` é necessário para acessar o índice correto
            
            # Ângulos dos cotovelos (ombro, cotovelo, punho)
            left_elbow_angle = calculate_angle(
                [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x, landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y],
                [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x, landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y],
                [landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x, landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y]
            )
            right_elbow_angle = calculate_angle(
                [landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y],
                [landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].y],
                [landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].y]
            )

            # Ângulos dos ombros (quadril, ombro, cotovelo)
            left_shoulder_angle = calculate_angle(
                [landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x, landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y],
                [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x, landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y],
                [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x, landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y]
            )
            right_shoulder_angle = calculate_angle(
                [landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].y],
                [landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y],
                [landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].y]
            )
            
            # Ângulos dos joelhos (quadril, joelho, tornozelo)
            left_knee_angle = calculate_angle(
                [landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x, landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y],
                [landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].x, landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].y],
                [landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].x, landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].y]
            )
            right_knee_angle = calculate_angle(
                [landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].y],
                [landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].y],
                [landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].y]
            )

            # Ângulos dos quadris (ombro, quadril, joelho) - essencial para postura de tronco
            left_hip_angle = calculate_angle(
                [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x, landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y],
                [landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x, landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y],
                [landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].x, landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].y]
            )
            right_hip_angle = calculate_angle(
                [landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y],
                [landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].y],
                [landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].y]
            )
            
            # Coletar todas as features em um vetor
            features = [
                left_elbow_angle, right_elbow_angle,
                left_shoulder_angle, right_shoulder_angle,
                left_knee_angle, right_knee_angle,
                left_hip_angle, right_hip_angle
            ]
            
            return np.array(features)
        else:
            return None # Nenhuma pose detectada
    except Exception as e:
        # print(f"❌ Erro ao extrair features de {image_path}: {e}") # Comentado para evitar poluir o log
        return None

# Nova função para gerar formas incorretas sinteticamente
def synthesize_incorrect_form(original_features, exercise_type_str):
    """
    Aplica uma perturbação sintética a uma cópia do array de features original
    para simular uma forma incorreta comum para o tipo de exercício especificado.

    Args:
        original_features (np.ndarray): O array de features (ângulos) da pose original (correta).
        exercise_type_str (str): O nome do exercício (ex: 'squat', 'deadlift', 'bicep curl').

    Returns:
        np.ndarray: O array de features modificado para simular a forma incorreta.
    """
    modified_features = original_features.copy()
    
    # Magnitude da perturbação angular (em graus)
    deviation_magnitude = np.random.uniform(15, 35) # Desvio de 15 a 35 graus

    if exercise_type_str == 'squat':
        # Erro comum: Não ir fundo o suficiente (joelhos não dobram o bastante)
        # Aumentar o ângulo do joelho para simular falta de profundidade
        modified_features[4] = np.clip(modified_features[4] + deviation_magnitude, 0, 180) # Left knee
        modified_features[5] = np.clip(modified_features[5] + deviation_magnitude, 0, 180) # Right knee
        
    elif exercise_type_str == 'deadlift':
        # Erro comum: Costas arredondadas (quadril muito baixo ou muito reto)
        # Diminuir o ângulo do quadril para simular um tronco menos inclinado para frente ou mais arredondado
        modified_features[6] = np.clip(modified_features[6] - deviation_magnitude, 0, 180) # Left hip
        modified_features[7] = np.clip(modified_features[7] - deviation_magnitude, 0, 180) # Right hip
        
    elif exercise_type_str == 'bicep curl':
        # Erro comum: Extensão incompleta (não esticar totalmente o braço na parte inferior)
        # Manter o ângulo do cotovelo maior do que deveria estar na extensão máxima
        modified_features[0] = np.clip(modified_features[0] - deviation_magnitude, 0, 180) # Left elbow (make it less straight)
        modified_features[1] = np.clip(modified_features[1] - deviation_magnitude, 0, 180) # Right elbow (make it less straight)
        
    # Para outros exercícios não especificados, o desvio não será aplicado.
    return modified_features

print("✅ Funções de extração de pose, cálculo de ângulos e geração de erros sintéticos prontas.")

# 4. Baixar e Pré-processar o Dataset de Exercícios (Com Geração de Erros)

# Definir o nome do dataset do Kaggle para exercícios
DATASET_SLUG = "hasyimabdillah/workoutexercises-images"
DATASET_NAME = "workout-exercises-images" # Nome da pasta onde será extraído
DATASET_PATH = Path(DATASET_NAME) # Caminho local

# Função para baixar e extrair o dataset do Kaggle
def download_exercise_dataset(dataset_slug, target_path):
    if not target_path.exists():
        print(f"⬇️ Baixando {dataset_slug} do Kaggle...")
        try:
            # Comando Kaggle para baixar e descompactar
            # O subprocess.run precisa que o comando 'kaggle' esteja acessível no PATH do ambiente
            subprocess.run(["kaggle", "datasets", "download", "-d", dataset_slug, "-p", str(target_path.parent), "--unzip"], check=True)
            print(f"✅ {dataset_slug} baixado e extraído para {target_path.parent}!")
            
            # O Kaggle pode baixar para uma pasta com o nome do slug, vamos renomear se necessário
            # e garantir que a estrutura seja a esperada.
            downloaded_dir = target_path.parent / dataset_slug.split('/')[-1]
            if downloaded_dir.exists() and downloaded_dir != target_path:
                print(f"Renomeando '{downloaded_dir}' para '{target_path}'...")
                shutil.move(downloaded_dir, target_path)
            
        except subprocess.CalledProcessError as e:
            print(f"❌ Erro ao baixar {dataset_slug} do Kaggle: {e}")
            print("Por favor, verifique se suas credenciais do Kaggle estão corretas e se você tem permissão para baixar o dataset.")
            print("Você pode precisar aceitar as regras do dataset no Kaggle primeiro: https://www.kaggle.com/datasets/hasyimabdillah/workoutexercises-images/code")
            return False
        except Exception as e:
            print(f"❌ Erro inesperado ao baixar {dataset_slug}: {e}")
            return False
    else:
        print(f"ℹ️ Dataset '{DATASET_NAME}' já existe em {target_path}. Pulando download.")
    return True

# Baixar o dataset de exercícios
dataset_downloaded = download_exercise_dataset(DATASET_SLUG, DATASET_PATH)

if not dataset_downloaded:
    print("❌ Não foi possível continuar sem o dataset. Encerrando o script.")
    sys.exit(1)


# Listar os tipos de exercícios (subpastas dentro do dataset)
exercise_types = [d.name for d in DATASET_PATH.iterdir() if d.is_dir()]
print(f"\n✅ Tipos de exercícios encontrados no dataset: {exercise_types}")

# Processamento dos Dados de Exercícios: Extrair Features e Gerar Rótulos de Forma (Correta/Incorreta)

# Listas para armazenar as features e os rótulos
all_features = []
all_exercise_labels = []
all_form_labels = [] # 0 para correto, 1 para incorreto

# Inicializar o objeto MediaPipe Pose para usar na extração de features
with mp_pose.Pose(static_image_mode=True, min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose_detector:
    for exercise_type in exercise_types:
        exercise_dir = DATASET_PATH / exercise_type
        image_files = [f for f in exercise_dir.iterdir() if f.suffix.lower() in ('.jpg', '.jpeg', '.png')]
        
        print(f"\n⚙️ Processando {len(image_files)} imagens para o exercício: {exercise_type}")
        
        # Amostragem para limitar o tamanho do dataset e acelerar o processamento durante o desenvolvimento.
        # Reduzir este número acelerará o tempo de pré-processamento e treinamento.
        SAMPLE_PER_EXERCISE = 200 # Processar até 200 imagens por tipo de exercício para um teste rápido
        
        if len(image_files) > SAMPLE_PER_EXERCISE:
            # Use list() para garantir que a amostra seja uma lista de caminhos, não um array numpy
            image_files_sampled = np.random.choice(image_files, SAMPLE_PER_EXERCISE, replace=False).tolist()
            print(f"   Amostrando {SAMPLE_PER_EXERCISE} imagens de {len(image_files)} disponíveis para {exercise_type}.")
        else:
            image_files_sampled = image_files.tolist() # Se menos que a amostra, usar todas.

        # Definir a proporção de exemplos "incorretos" que queremos gerar sinteticamente
        # Isso ajuda a balancear o dataset para as classes "correta" e "incorreta"
        synthetic_incorrect_ratio = 0.4 # Queremos que 40% das amostras processadas sejam sinteticamente incorretas
        num_samples_to_make_incorrect = int(len(image_files_sampled) * synthetic_incorrect_ratio)
        
        # Selecionar aleatoriamente os índices das imagens onde aplicaremos o erro sintético
        # Certifique-se de que não tentamos selecionar mais índices do que amostras disponíveis
        indices_to_make_incorrect = np.random.choice(
            len(image_files_sampled),
            min(num_samples_to_make_incorrect, len(image_files_sampled)),
            replace=False
        )
        
        for i, img_file in enumerate(tqdm(image_files_sampled, desc=f"   Extraindo features de {exercise_type}")):
            original_features = extract_pose_features(str(img_file), pose_detector)
            
            if original_features is not None:
                # Decide if this sample should be made synthetically incorrect
                if i in indices_to_make_incorrect:
                    # Apply synthetic error and label as incorrect
                    features_to_add = synthesize_incorrect_form(original_features, exercise_type)
                    form_status = 1 # Incorreto
                else:
                    # Keep original features and label as correct
                    features_to_add = original_features
                    form_status = 0 # Correto

                all_features.append(features_to_add)
                all_exercise_labels.append(exercise_type)
                all_form_labels.append(form_status)
            else:
                pass # Already prints error if image not loaded or pose not detected


# Verificar se há dados para processar
if not all_features:
    print("\n❌ Nenhuma feature foi extraída. Verifique seu dataset e a detecção de pose.")
    print("O script será encerrado pois não há dados para treinar o modelo.")
    sys.exit(1) # Encerrar o script

# Converter listas para arrays NumPy
X = np.array(all_features)
y_exercise_raw = np.array(all_exercise_labels)
y_form = np.array(all_form_labels)

# Codificar rótulos de exercício e forma para formato numérico (one-hot encoding)
# Para exercícios, precisamos de um LabelEncoder primeiro para mapear strings para inteiros.
exercise_encoder = LabelEncoder()
y_exercise_encoded = exercise_encoder.fit_transform(y_exercise_raw)
y_exercise = to_categorical(y_exercise_encoded)

# Para a forma, já temos 0/1, então podemos diretamente para one-hot se necessário.
# Como é binário (0/1), to_categorical é útil para a função de perda.
y_form = to_categorical(y_form, num_classes=2)

print(f"\n✅ Total de {len(X)} amostras processadas.")
print(f"   Shape das features (X): {X.shape}")
print(f"   Shape dos rótulos de exercício (y_exercise): {y_exercise.shape}")
print(f"   Shape dos rótulos de forma (y_form): {y_form.shape}")
print(f"   Exercícios detectados: {exercise_encoder.classes_}")

# Salvar o LabelEncoder para uso posterior na inferência
np.save('exercise_encoder.npy', exercise_encoder)

# Normalizar as features (importante para redes neurais)
# Usaremos MinMaxScaler para escalar as features entre 0 e 1.
feature_scaler = MinMaxScaler()
X_scaled = feature_scaler.fit_transform(X)
np.save('feature_scaler.npy', feature_scaler)

print("✅ Pré-processamento de dados de exercícios concluído e escalers salvos.")

# 5. Construção e Treinamento do Modelo de Detecção de Forma

# Caminho para salvar o modelo
MODEL_PATH = 'gym_form_detector_model.h5'

# Verificar se um modelo pré-treinado já existe
if os.path.exists(MODEL_PATH):
    print("ℹ️ Modelo pré-treinado encontrado. Carregando...")
    # O TensorFlow irá automaticamente carregar e usar a CPU se nenhuma GPU estiver disponível
    model = load_model(MODEL_PATH)
    
    # Carregar scalers para consistência (necessário mesmo se o modelo já existe)
    exercise_encoder = np.load('exercise_encoder.npy', allow_pickle=True).item()
    feature_scaler = np.load('feature_scaler.npy', allow_pickle=True).item()
    print("✅ Modelo pré-existente e scalers carregados.")
else:
    print("⚙️ Modelo não encontrado. Iniciando construção e treinamento de novo modelo...")

    # Definir a arquitetura do modelo
    def build_exercise_form_model(input_shape, num_exercise_classes):
        # A entrada agora é um vetor de features (ângulos e distâncias)
        input_layer = Input(shape=(input_shape,))
        
        # Camadas densas para processar as features
        x = Dense(512, activation='relu')(input_layer)
        x = BatchNormalization()(x) # Ajuda na estabilidade do treinamento
        x = Dropout(0.4)(x) # Previne overfitting
        
        x = Dense(256, activation='relu')(x)
        x = BatchNormalization()(x)
        x = Dropout(0.3)(x)
        
        x = Dense(128, activation='relu')(x)
        x = BatchNormalization()(x)
        x = Dropout(0.2)(x)
        
        # Duas saídas para as duas tarefas de classificação
        exercise_output = Dense(num_exercise_classes, activation='softmax', name='exercise_output')(x)
        form_output = Dense(2, activation='softmax', name='form_output')(x) # 2 classes: correto/incorreto
        
        model = Model(inputs=input_layer, outputs=[exercise_output, form_output])
        return model

    # Criar o modelo
    num_features = X_scaled.shape[1] # Número de features extraídas
    num_exercise_classes = y_exercise.shape[1] # Número de tipos de exercícios
    model = build_exercise_form_model(num_features, num_exercise_classes)
    
    # Compilar o modelo
    # Usamos 'categorical_crossentropy' para ambas as saídas, pois são problemas de classificação multi-classe.
    # Os 'loss_weights' permitem balancear a importância de cada tarefa durante o treinamento.
    model.compile(
        optimizer=Adam(learning_rate=0.001),
        loss={
            'exercise_output': 'categorical_crossentropy',
            'form_output': 'categorical_crossentropy'
        },
        loss_weights={
            'exercise_output': 0.7, # Maior peso para classificar o exercício corretamente
            'form_output': 0.3 # Menor peso, mas ainda importante para a forma
        },
        metrics={
            'exercise_output': ['accuracy'],
            'form_output': ['accuracy']
        }
    )
    
    model.summary()

    # Callbacks para um treinamento mais robusto
    checkpoint = ModelCheckpoint(
        MODEL_PATH,
        save_best_only=True,
        monitor='val_loss', # Monitorar a perda de validação combinada
        verbose=1
    )
    reduce_lr = ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.5, # Reduz a LR pela metade
        patience=5, # Se val_loss não melhorar por 5 épocas
        min_lr=1e-7,
        verbose=1
    )
    early_stop = EarlyStopping(
        monitor='val_loss',
        patience=15, # Para o treinamento se val_loss não melhorar por 15 épocas
        restore_best_weights=True, # Restaura os pesos do melhor modelo
        verbose=1
    )
    
    # Dividir os dados em conjuntos de treino e validação
    # Usamos o mesmo conjunto de features X_scaled para ambas as saídas.
    X_train, X_val, y_exercise_train, y_exercise_val, y_form_train, y_form_val = train_test_split(
        X_scaled, y_exercise, y_form, test_size=0.2, random_state=42
    )

    # Treinar o modelo
    # Passamos os dicionários de labels para o fit.
    # O treinamento ocorrerá na CPU.
    history = model.fit(
        X_train,
        {'exercise_output': y_exercise_train, 'form_output': y_form_train},
        epochs=100, # Um número razoável de épocas, EarlyStopping vai parar antes se convergir
        batch_size=32,
        validation_data=(X_val, {'exercise_output': y_exercise_val, 'form_output': y_form_val}),
        callbacks=[checkpoint, reduce_lr, early_stop],
        verbose=1
    )
    
    print(f"✅ Modelo treinado e salvo em {MODEL_PATH}")

# 6. Detecção de Forma de Exercícios em Tempo Real

# Carregar o modelo e os scalers/encoders se não foram carregados anteriormente
if 'model' not in locals():
    print("ℹ️ Carregando modelo pré-existente para detecção em tempo real...")
    # O TensorFlow irá carregar e usar a CPU.
    model = load_model(MODEL_PATH)
    exercise_encoder = np.load('exercise_encoder.npy', allow_pickle=True).item()
    feature_scaler = np.load('feature_scaler.npy', allow_pickle=True).item()
    print("✅ Modelo e scalers carregados.")

# Mapeamento para os rótulos de forma
form_labels = ['Forma Correta', 'Forma Incorreta']

# Inicializar o MediaPipe Pose para detecção em tempo real
# 'static_image_mode=False' para melhor desempenho em streams de vídeo.
# O MediaPipe automaticamente usa a CPU neste ambiente.
pose_live_detector = mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5)

# Função de predição em tempo real
def detect_exercise_form(frame, pose_detector, model, feature_scaler, exercise_encoder, form_labels):
    """
    Processa um frame da webcam para detectar a pose e prever o exercício e a forma.
    Args:
        frame (numpy.ndarray): O frame de vídeo da webcam.
        pose_detector (mp.solutions.pose.Pose): Objeto MediaPipe Pose inicializado (executando em CPU).
        model (tensorflow.keras.Model): O modelo de ML treinado (inferência em CPU).
        feature_scaler (MinMaxScaler): O scaler usado para normalizar as features.
        exercise_encoder (LabelEncoder): O encoder usado para os rótulos de exercício.
        form_labels (list): Lista de rótulos para a forma (e.g., ['Correto', 'Incorreto']).
    Returns:
        numpy.ndarray: O frame com as informações de pose e feedback.
    """
    # Converter o frame para RGB para o MediaPipe
    image_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    
    # Melhorar o desempenho (opcional): marcar a imagem como não gravável para passar por referência.
    image_rgb.flags.writeable = False
    
    # Processar o frame para estimativa de pose (executa na CPU)
    results = pose_detector.process(image_rgb)
    
    # Marcar a imagem como gravável novamente para desenhar as anotações
    image_rgb.flags.writeable = True
    image_bgr = cv2.cvtColor(image_rgb, cv2.COLOR_RGB2BGR) # Voltar para BGR para o OpenCV
    
    feedback_text = "Nenhuma pose detectada."
    text_color = (255, 255, 255) # Cor padrão branca para feedback

    if results.pose_landmarks:
        # Desenhar as landmarks da pose no frame
        mp_drawing.draw_landmarks(
            image_bgr,
            results.pose_landmarks,
            mp_pose.POSE_CONNECTIONS,
            mp_drawing_styles.get_default_pose_landmarks_style()
        )
        
        # Extrair features da pose detectada no frame atual
        # Replicamos a lógica de extract_pose_features para o frame atual
        landmarks = results.pose_landmarks.landmark
        
        try:
            # Coletar as mesmas features que foram usadas no treinamento
            live_features = np.array([
                calculate_angle(
                    [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x, landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y],
                    [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x, landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y],
                    [landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x, landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y]
                ),
                calculate_angle(
                    [landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y],
                    [landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].y],
                    [landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].y]
                ),
                calculate_angle(
                    [landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x, landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y],
                    [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x, landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y],
                    [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x, landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y]
                ),
                calculate_angle(
                    [landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].y],
                    [landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y],
                    [landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].y]
                ),
                calculate_angle(
                    [landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x, landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y],
                    [landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].x, landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].y],
                    [landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].x, landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].y]
                ),
                calculate_angle(
                    [landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].y],
                    [landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].y],
                    [landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].y]
                ),
                calculate_angle(
                    [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x, landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y],
                    [landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x, landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y],
                    [landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].x, landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].y]
                ),
                calculate_angle(
                    [landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y],
                    [landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].y],
                    [landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].y]
                )
            ])
            
            # Normalizar as features usando o scaler treinado
            live_features_scaled = feature_scaler.transform(live_features.reshape(1, -1))
            
            # Fazer a predição com o modelo (inferência em CPU)
            predictions = model.predict(live_features_scaled, verbose=0)
            
            # Interpretar as predições
            exercise_probs = predictions[0][0]
            form_probs = predictions[1][0]
            
            # Obter o exercício e a forma com maior probabilidade
            predicted_exercise_idx = np.argmax(exercise_probs)
            predicted_exercise = exercise_encoder.inverse_transform([predicted_exercise_idx])[0]
            
            predicted_form_idx = np.argmax(form_probs)
            predicted_form = form_labels[predicted_form_idx]
            
            feedback_text = f"Exercicio: {predicted_exercise} ({exercise_probs[predicted_exercise_idx]*100:.1f}%) | Forma: {predicted_form} ({form_probs[predicted_form_idx]*100:.1f}%)"
            
            # Adicionar sugestão de correção se a forma for incorreta (exemplo genérico)
            if predicted_form_idx == 1: # Se a forma for incorreta
                text_color = (0, 0, 255) # Vermelho
                feedback_text += " -> Ajuste sua postura!"
            else:
                text_color = (0, 255, 0) # Verde para correto

        except Exception as e:
            feedback_text = f"Erro na predição: {e}"
            text_color = (0, 165, 255) # Laranja para erro
            print(f"Erro na detecção: {e}")

    # Exibir o feedback no frame
    cv2.putText(image_bgr, feedback_text, (10, 30),
                cv2.FONT_HERSHEY_SIMPLEX, 0.8, text_color, 2, cv2.LINE_AA)
    
    return image_bgr

# Loop da Webcam para Detecção em Tempo Real

# Inicializar a captura de vídeo da webcam (0 é geralmente a câmera padrão)
cap = cv2.VideoCapture(0)

if not cap.isOpened():
    print("❌ Não foi possível acessar a webcam. Verifique se ela está conectada e não está em uso.")
else:
    # Definir resolução da webcam para melhor qualidade
    cap.set(cv2.CAP_PROP_FRAME_WIDTH, 1280)
    cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 720)
    
    window_name = 'Detecao de Forma de Exercicios (CPU) - Pressione Q para Sair'
    cv2.namedWindow(window_name, cv2.WINDOW_NORMAL)
    
    print("\n✅ Webcam ativada. Pressione 'Q' para sair da janela de visualização.")
    # Exibir a hora atual conforme as instruções
    current_utc_time = time.gmtime()
    current_brasilia_time = time.localtime(time.time() - 3 * 3600) # UTC-3
    print(f"UTC: {time.strftime('%d/%m/%Y %H:%M:%S', current_utc_time)} (UTC)")
    print(f"Brasília: {time.strftime('%d/%m/%Y %H:%M:%S', current_brasilia_time)} (UTC-3)")
    print("\n⚠️ A performance pode ser limitada pela sua CPU. Para melhor experiência, considere GPUs dedicadas.")

    try:
        while True:
            ret, frame = cap.read()
            if not ret:
                print("❌ Falha ao ler frame da webcam.")
                break
                
            # Espelhar o frame para que a visualização seja mais intuitiva (como um espelho)
            frame = cv2.flip(frame, 1)
            
            # Chamar a função de detecção e obter o frame com feedback
            output_frame = detect_exercise_form(frame, pose_live_detector, model, feature_scaler, exercise_encoder, form_labels)
            
            # Mostrar o frame na janela
            cv2.imshow(window_name, output_frame)
            
            # Verificar se a janela foi fechada pelo usuário
            if cv2.getWindowProperty(window_name, cv2.WND_PROP_VISIBLE) < 1:
                break
                
            # Sair do loop se a tecla 'q' for pressionada
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break
    
    finally:
        # Liberar os recursos da webcam e fechar todas as janelas do OpenCV
        cap.release()
        cv2.destroyAllWindows()
        print("✅ Programa finalizado. Recursos liberados.")


ℹ️ Nenhuma GPU NVIDIA detectada ou configurada. Todas as operações de Machine Learning utilizarão a CPU.
✅ Ambiente configurado. Credenciais do Kaggle salvas e ambiente preparado para CPU.
✅ Funções de extração de pose, cálculo de ângulos e geração de erros sintéticos prontas.
ℹ️ Dataset 'workout-exercises-images' já existe em workout-exercises-images. Pulando download.

✅ Tipos de exercícios encontrados no dataset: ['barbell biceps curl', 'bench press', 'chest fly machine', 'deadlift', 'decline bench press', 'hammer curl', 'hip thrust', 'incline bench press', 'lat pulldown', 'lateral raises', 'leg extension', 'leg raises', 'plank', 'pull up', 'push up', 'romanian deadlift', 'russian twist', 'shoulder press', 'squat', 't bar row', 'tricep dips', 'tricep pushdown']

⚙️ Processando 705 imagens para o exercício: barbell biceps curl
   Amostrando 200 imagens de 705 disponíveis para barbell biceps curl.


   Extraindo features de barbell biceps curl: 100%|██████████████████████████████████| 200/200 [00:10<00:00, 18.68it/s]



⚙️ Processando 625 imagens para o exercício: bench press
   Amostrando 200 imagens de 625 disponíveis para bench press.


   Extraindo features de bench press: 100%|██████████████████████████████████████████| 200/200 [00:10<00:00, 19.65it/s]



⚙️ Processando 527 imagens para o exercício: chest fly machine
   Amostrando 200 imagens de 527 disponíveis para chest fly machine.


...


Epoch 68: ReduceLROnPlateau reducing learning rate to 3.125000148429535e-05.
98/98 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - exercise_output_accuracy: 0.6017 - exercise_output_loss: 1.2548 - form_output_accuracy: 0.6147 - form_output_loss: 0.6688 - loss: 1.0790 - val_exercise_output_accuracy: 0.5921 - val_exercise_output_loss: 1.3918 - val_form_output_accuracy: 0.6049 - val_form_output_loss: 0.6735 - val_loss: 1.1780 - learning_rate: 6.2500e-05
Epoch 69/100
87/98 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - exercise_output_accuracy: 0.5680 - exercise_output_loss: 1.3245 - form_output_accuracy: 0.6065 - form_output_loss: 0.6654 - loss: 1.1267
Epoch 69: val_loss improved from 1.17802 to 1.17778, saving model to gym_form_detector_model.h5


98/98 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - exercise_output_accuracy: 0.5690 - exercise_output_loss: 1.3221 - form_output_accuracy: 0.6071 - form_output_loss: 0.6651 - loss: 1.1250 - val_exercise_output_accuracy: 0.5959 - val_exercise_output_loss: 1.3911 - val_form_output_accuracy: 0.6049 - val_form_output_loss: 0.6736 - val_loss: 1.1778 - learning_rate: 3.1250e-05
Epoch 70/100
97/98 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - exercise_output_accuracy: 0.5743 - exercise_output_loss: 1.2833 - form_output_accuracy: 0.6047 - form_output_loss: 0.6686 - loss: 1.0989
Epoch 70: val_loss did not improve from 1.17778
98/98 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - exercise_output_accuracy: 0.5742 - exercise_output_loss: 1.2835 - form_output_accuracy: 0.6048 - form_output_loss: 0.6685 - loss: 1.0990 - val_exercise_output_accuracy: 0.5921 - val_exercise_output_loss: 1.3944 - val_form_output_accuracy: 0.6036 - val_form_output_loss: 0.6739 - val_loss: 1.1798 - learning_rate: 3.1250e-05
Epoch 71/100
87/98 ━━━━━━━━━━━━

98/98 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - exercise_output_accuracy: 0.5683 - exercise_output_loss: 1.3088 - form_output_accuracy: 0.6147 - form_output_loss: 0.6627 - loss: 1.1149 - val_exercise_output_accuracy: 0.5972 - val_exercise_output_loss: 1.3905 - val_form_output_accuracy: 0.6049 - val_form_output_loss: 0.6732 - val_loss: 1.1775 - learning_rate: 1.5625e-05
Epoch 77/100
95/98 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - exercise_output_accuracy: 0.5813 - exercise_output_loss: 1.2770 - form_output_accuracy: 0.6030 - form_output_loss: 0.6687 - loss: 1.0945
Epoch 77: val_loss improved from 1.17750 to 1.17708, saving model to gym_form_detector_model.h5


98/98 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - exercise_output_accuracy: 0.5810 - exercise_output_loss: 1.2774 - form_output_accuracy: 0.6032 - form_output_loss: 0.6685 - loss: 1.0947 - val_exercise_output_accuracy: 0.5972 - val_exercise_output_loss: 1.3902 - val_form_output_accuracy: 0.6036 - val_form_output_loss: 0.6731 - val_loss: 1.1771 - learning_rate: 1.5625e-05
Epoch 78/100
95/98 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - exercise_output_accuracy: 0.5791 - exercise_output_loss: 1.3032 - form_output_accuracy: 0.6112 - form_output_loss: 0.6613 - loss: 1.1106
Epoch 78: val_loss did not improve from 1.17708
98/98 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - exercise_output_accuracy: 0.5792 - exercise_output_loss: 1.3025 - form_output_accuracy: 0.6111 - form_output_loss: 0.6614 - loss: 1.1101 - val_exercise_output_accuracy: 0.5934 - val_exercise_output_loss: 1.3908 - val_form_output_accuracy: 0.6049 - val_form_output_loss: 0.6730 - val_loss: 1.1773 - learning_rate: 1.5625e-05
Epoch 79/100
89/98 ━━━━━━━━━━━━

98/98 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - exercise_output_accuracy: 0.5758 - exercise_output_loss: 1.2694 - form_output_accuracy: 0.6301 - form_output_loss: 0.6578 - loss: 1.0859 - val_exercise_output_accuracy: 0.5959 - val_exercise_output_loss: 1.3899 - val_form_output_accuracy: 0.6061 - val_form_output_loss: 0.6731 - val_loss: 1.1766 - learning_rate: 1.5625e-05
Epoch 82/100
90/98 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - exercise_output_accuracy: 0.5641 - exercise_output_loss: 1.3097 - form_output_accuracy: 0.6240 - form_output_loss: 0.6539 - loss: 1.1130
Epoch 82: val_loss did not improve from 1.17662
98/98 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - exercise_output_accuracy: 0.5646 - exercise_output_loss: 1.3084 - form_output_accuracy: 0.6229 - form_output_loss: 0.6547 - loss: 1.1123 - val_exercise_output_accuracy: 0.5934 - val_exercise_output_loss: 1.3915 - val_form_output_accuracy: 0.6061 - val_form_output_loss: 0.6731 - val_loss: 1.1779 - learning_rate: 1.5625e-05
Epoch 83/100
89/98 ━━━━━━━━━━━━